In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv('elspeth_data.csv')

In [4]:
import re
import csv
import hashlib
import numpy as np
import pandas as pd

# ============================================================
# FINAL FROZEN PIPELINE (curated mass-casualty refs)
# ============================================================
# Goal:
# - Build ONOMAP-ready CSV with:
#     Unique Identifier (10-digit), Forename, Surname
# - Handle true mass-casualty events by extracting individual victim names
#   from PDF text (Option A) and excluding event placeholder “names”.
# - Keep surname-only individuals (e.g. "Pether") as Surname with blank Forename.
#
# Assumes df is your FULL dataframe with at least:
#   - ref
#   - name_of_deceased
#   - text   (full extracted PDF text)
#
# Curated (manually verified) mass-casualty refs:
#   - 2017-0206 (Sousse terror attack)
#   - 2018-0262 (Grenfell Tower fire)
#   - 2019-0332 (London Bridge terror attack)
# ============================================================



# ----------------------------
# 0) Curated mass-casualty refs (manual truth set)
# ----------------------------
MASS_CASUALTY_REFS = {
    "2017-0206",  # Sousse
    "2018-0262",  # Grenfell Tower
    "2019-0332",  # London Bridge
}


# ----------------------------
# 1) Curated victim lists for mass-casualty events (manual truth)
# ----------------------------
MASS_EVENT_VICTIMS = {
    "2017-0206": [  # Sousse
        "Christopher Bell",
        "Sharon Bell",
        "Lisa Burbidge",
        "Scott Chalkley",
        "Stuart Cullen",
        "Suzanne Davey",
        "Christopher Dyer",
        "Adrian Evans",
        "Charles Evans",
        "Angela Fisher",
        "Raymond Fisher",
        "Lisa Graham",
        "William Graham",
        "Philip Heathcote",
        "Trudy Jones",
        "Carly Lovett",
        "Ann McQuire",
        "James McQuire",
        "Stephen Mellor",
        "Joel Richards",
        "John Stollery",
        "Janet Stocker",
        "John Stocker",
        "Eileen Swannack",
        "David Thompson",
        "Denis Thwaites",
        "Elaine Thwaites",
        "John Welch",
        "Bruce Wilkinson",
        "Claire Windass",
    ],
    "2018-0262": [ # Grenfell
    "Fatemeh Afraisabi",
    "Sakina Afraisabi",
    "Fathia Ahmed",
    "Amal Ahmedin",
    "Mohammad Al Haj Ali",
    "Alexandra Atalla",
    "Husnia Begum",
    "Rabeya Begum",
    "Leena Belkadi",
    "Malak Belkadi",
    "Omar Belkadi",
    "Raymond Bernard",
    "Vincent Chiejina",
    "Fatima Choucair",
    "Nadia Choucair",
    "Siria Choucair",
    "Bassem Choucair",
    "Mierna Choucair",
    "Zainab Choucair",
    "Joseph Daniels",
    "Jeremiah Deen",
    "Zainab Deen",
    "Anthony Disson",
    "Eslah Elgwahry",
    "Mariem Elgwahry",
    "Abdulaziz El-Wahabi",
    "Faouzia El-Wahabi",
    "Mehdi El-Wahabi",
    "Nur Huda El-Wahabi",
    "Yasin El-Wahabi",
    "Marco Gottardi",
    "Berkat Haffar",
    "Rinak Haffar",
    "Farah Hamdan",
    "Mohammed Hamid",
    "Mohammed Hanif",
    "Firdaws Hashim",
    "Yaqub Hashim",
    "Yahya Hashim",
    "Fethia Hessen",
    "Hania Hassan",
    "Abrarles Mohamed Ibrahim",
    "Ira Ibrahim",
    "Rania Ibrahim",
    "Amna Mahmud Idris",
    "Ali Yawar Jafri",
    "Nura Jemsil",
    "Hamid Kani",
    "Hashim Kedir",
    "Khadija Khalloufi",
    "Victoria King",
    "Deborah Lamprell",
    "Gary Maunders",
    "Mary Mendy",
    "Karru Miah",
    "Liqoya Moore",
    "Dennis Murphy",
    "Mohammed Amied Neda",
    "Isaac Paulos",
    "Steven Power",
    "Hesham Rahman",
    "Khadija Saye",
    "Abdeslam Sebbar",
    "Sheila Smith",
    "Gloria Trevisan",
    "Amaya Tuocu-Ahmedin",
    "Mohamednur Tuczu",
    "Jessica Urbano-Ramirez",
    "Marjorie Vital",
    "Ernie Vital",
    "Baby Logan Gomes",
    ],
    "2019-0332": [  # London Bridge
        "Xavier Thomas",
        "Christine Archibald",
        "James McMullan",
        "Alexandre Pigeard",
        "Kirsty Boden",
        "Sébastien Bélanger",
        "Sara Zelenak",
        "Ignacio Echeverria Miralles De Imperial",
    ],
}

# ----------------------------
# 1) Build mass-casualty people dataframe from curated lists
# ----------------------------
event_people = []

for ref, names in MASS_EVENT_VICTIMS.items():
    for name in names:
        event_people.append({
            "ref": ref,
            "name_of_deceased": name
        })

mass_event_people_df = (
    pd.DataFrame(event_people)
    .drop_duplicates()
    .reset_index(drop=True)
)

print("Mass-casualty victims loaded:")
print(mass_event_people_df.groupby("ref")["name_of_deceased"].count())


# ----------------------------
# 2) Build combined names dataframe:
#    - non-event refs: use df['name_of_deceased']
#    - curated event refs: use extracted victim list (NOT event placeholder)
# ----------------------------
df_non_event_names = df.loc[
    ~df["ref"].astype(str).isin(MASS_CASUALTY_REFS),
    ["ref", "name_of_deceased"]
].copy()

names_df = pd.concat([df_non_event_names, mass_event_people_df], ignore_index=True)

# Drop blanks early
names_df = names_df[
    names_df["name_of_deceased"].notna() &
    (names_df["name_of_deceased"].astype(str).str.strip() != "")
].copy()

# ----------------------------
# 3) Shared-surname expansion (handles and/& and comma-lists)
# ----------------------------
JOINER = re.compile(r'(?i)\b(?:and)\b|&')

def expand_shared_surname(x) -> str:
    if x is None:
        return x
    s = str(x).strip()
    if not s:
        return s

    # normalise unicode punctuation
    s = (s.replace("\u2018", "'").replace("\u2019", "'")
           .replace("\u2013", "-").replace("\u2014", "-"))
    s = re.sub(r"\s+", " ", s).strip()

    if not JOINER.search(s):
        return s

    # normalise sluggy joiners near and/& (e.g. Bryan-and-Mary-Andrews)
    s_near = re.sub(r'(?i)\s*[-_]+\s*(and|&)\s*[-_]+\s*', r' \1 ', s)

    # Case: "A, B and C Surname" or "A, B & C Surname"
    m_list = re.match(
        r"^\s*(?P<list>(?:[A-Za-zÀ-ÖØ-öø-ÿ'’\-\.]+\s*,\s*)+)"
        r"(?P<last>[A-Za-zÀ-ÖØ-öø-ÿ'’\-\.]+)\s+(?:and|&)\s+"
        r"(?P<final>[A-Za-zÀ-ÖØ-öø-ÿ'’\-\.]+)\s+"
        r"(?P<surname>.+?)\s*$",
        s_near
    )
    if m_list:
        list_part = m_list.group("list")
        last_in_list = m_list.group("last")
        final_forename = m_list.group("final")
        surname = m_list.group("surname").strip()

        # guardrails
        if re.search(r"\d", surname):
            return s
        if not re.match(r"^[A-Za-zÀ-ÖØ-öø-ÿ'’\-\. ]+$", surname):
            return s

        forenames = [p.strip() for p in list_part.split(",") if p.strip()]
        forenames.append(last_in_list.strip())
        forenames.append(final_forename.strip())
        return ", ".join([f"{fn} {surname}" for fn in forenames])

    # Case: "A and B Surname" or "A & B Surname"
    m_two = re.match(
        r"^\s*([A-Za-zÀ-ÖØ-öø-ÿ'’\-\.]+)\s+(?:and|&)\s+([A-Za-zÀ-ÖØ-öø-ÿ'’\-\.]+)\s+(.+?)\s*$",
        s_near
    )
    if not m_two:
        return s

    a, b, surname = m_two.group(1).strip(), m_two.group(2).strip(), m_two.group(3).strip()
    if re.search(r"\d", surname):
        return s
    if not re.match(r"^[A-Za-zÀ-ÖØ-öø-ÿ'’\-\. ]+$", surname):
        return s
    return f"{a} {surname}, {b} {surname}"

names_df["name_of_deceased"] = names_df["name_of_deceased"].map(expand_shared_surname)

# ----------------------------
# 4) Clean + split/explode (single pass)
# ----------------------------
SPLIT_PAT = r"(?x),|;|\s+&\s+|\s+and\s+|(?:\s*[-–—_]\s*and\s*[-–—_]\s*)"

# Existing cut (requires separators/colons)
CUT_PAT = re.compile(r"""(?ix)
    (?:\s*(?:\||\n| - | — | – )\s*|
       \s*\b(?:coroner\s+name|coroner\s+area|category|sent\s+to|to|recipient)\s*:\s*
    )
    (?:coroner\s+name|coroner\s+area|category|this\s+report|regulation\s*28|
       report\s+is\s+being\s+sent|sent\s+to|recipient|to|nhs\s+trust|
       department\s+of\s+health|integrated\s+care|commission|council)\b
    .*$""")

# NEW: hard-cut even when metadata is glued to the surname (e.g. "WochnaCoroner name:")
META_CUT = re.compile(r"""(?ix)
    \b(
        coroner\s+name\s*:|
        coroner\s+area\s*:|
        category\s*:|
        this\s+report\s+is\s+being\s+sent\s+to\s*:|
        report\s+is\s+being\s+sent\s+to\s*:|
        sent\s+to\s*:|
        recipient\s*:
    ).*$
""")

names_df = (
    names_df.assign(
        name_of_deceased=(
            names_df["name_of_deceased"]
            .astype(str)

            # Unicode punctuation normalisation
            .str.replace("\u2018", "'", regex=False)
            .str.replace("\u2019", "'", regex=False)
            .str.replace("\u2013", "-", regex=False)
            .str.replace("\u2014", "-", regex=False)

            # ✅ Move CamelCase deglue EARLY (so "WochnaCoroner" becomes "Wochna Coroner")
            .str.replace(r"(?<=[a-z])(?=[A-Z])", " ", regex=True)

            # ✅ Hard-cut at metadata keywords (even if glued / no separators)
            .str.replace(META_CUT, "", regex=True)

            # Existing leading label removal
            .str.replace(r"(?i)^\s*(deceased|deceased names?)\s*[:\-]\s*", "", regex=True)

            # Existing cut (separator-based)
            .str.replace(CUT_PAT, "", regex=True)

            # Remove bracketed notes + titles
            .str.replace(r"\(.*?\)", "", regex=True)
            .str.replace(r"\[.*?\]", "", regex=True)
            .str.replace(r"(?i)\b(mr|mrs|ms|miss|dr|prof|sir|madam)\b\.?", "", regex=True)
            .str.replace(r"(?i)\b(the\s+late|late)\b", "", regex=True)

            # Whitespace tidy
            .str.replace(r"\s+", " ", regex=True)
            .str.strip()

            # Split multiple names
            .str.split(SPLIT_PAT, regex=True)
        )
    )
    .explode("name_of_deceased")
    .assign(
        name_of_deceased=lambda d: (
            d["name_of_deceased"]
            .astype(str)
            .str.replace(r"^[\s,;:.|\-–—]+|[\s,;:.|\-–—]+$", "", regex=True)
            .str.replace(r"\s+", " ", regex=True)
            .str.strip()
        )
    )
    .reset_index(drop=True)
)

# Drop obvious placeholders
names_df = names_df[
    names_df["name_of_deceased"].notna()
    & (names_df["name_of_deceased"].astype(str).str.strip() != "")
    & (~names_df["name_of_deceased"].str.match(r"(?i)^(unknown|not known|n/?a|unidentified)$", na=False))
].copy()

# ----------------------------
# 5) Split Forename/Surname
#    IMPORTANT: single token -> Surname (keeps Pether etc. for ONOMAP)
# ----------------------------
SURNAME_PARTICLES = {
    "al", "el",
    "da", "de", "del", "della", "der", "den",
    "di", "du",
    "la", "le", "lo",
    "van", "von",
    "bin", "ibn",
    "st", "st.", "saint",
    "ter",
}

def split_forename_surname(full: str):
    if full is None:
        return (np.nan, np.nan)
    s = str(full).strip()
    if not s:
        return (np.nan, np.nan)

    parts = s.split()
    if len(parts) == 1:
        return (np.nan, parts[0])  # surname-only

    i = len(parts) - 1
    surname_parts = [parts[i]]
    i -= 1
    while i >= 0 and parts[i].rstrip(".").lower() in SURNAME_PARTICLES:
        surname_parts.insert(0, parts[i])
        i -= 1

    surname = " ".join(surname_parts)
    forename = " ".join(parts[: i + 1]).strip() or np.nan
    return (forename, surname)

spl = names_df["name_of_deceased"].apply(split_forename_surname)
names_df["Forename"] = spl.map(lambda x: x[0])
names_df["Surname"] = spl.map(lambda x: x[1])

# Require surname for ONOMAP
names_df = names_df.dropna(subset=["Surname"]).copy()
names_df["surname_only"] = names_df["Forename"].isna() & names_df["Surname"].notna()

# ----------------------------
# 6) Deduplicate per (ref, Forename, Surname)
# ----------------------------
df_dedup = (
    names_df
    .drop_duplicates(subset=["ref", "Forename", "Surname"])
    .reset_index(drop=True)
)

# ----------------------------
# 7) Stable 10-digit Unique Identifier
# ----------------------------
def make_onomap_id(row) -> str:
    forename = "" if pd.isna(row["Forename"]) else row["Forename"]
    key = f"{row['ref']}|{forename}|{row['Surname']}"
    h = hashlib.sha256(key.encode("utf-8")).hexdigest()
    return str(int(h, 16))[:10]

df_dedup["Unique Identifier"] = df_dedup.apply(make_onomap_id, axis=1)

assert df_dedup["Unique Identifier"].is_unique, "Duplicate Unique Identifiers found"
assert df_dedup["Unique Identifier"].astype(str).str.match(r"^\d{10}$").all(), "ID format error"

# ----------------------------
# 8) Final exclusions (auditable, minimal)
# ----------------------------
# Rationale:
# - 'Sousse' is an event/location label, not an individual; we instead use victim names extracted from PDF text
# - 'Tower' is the Grenfell Tower event placeholder (Forename='Grenfell', Surname='Tower'); victims extracted from PDF text
# - 'Redacted', 'C', 'Yz' are administrative/non-informative tokens for ethnicity inference
# - 'M5', refers to M5 '7' https://www.judiciary.uk/wp-content/uploads/2015/01/M5-Seven-2014-0564.pdf - doesn't name the 7
exclude_surnames = {"Sousse", "Redacted", "REDACTED", "C", "Yz", "YZ", "Tower", "M5"}

df_dedup = df_dedup[~df_dedup["Surname"].isin(exclude_surnames)].copy()

# ----------------------------
# 9) Export ONOMAP input
# ----------------------------
onomap_df = df_dedup[["Unique Identifier", "Forename", "Surname"]].copy()
onomap_df["Unique Identifier"] = onomap_df["Unique Identifier"].astype(str)

onomap_df.to_csv(
    "pfd_onno_times_curated.csv",
    index=False,
    quoting=csv.QUOTE_NONNUMERIC
)

print("Wrote: pfd_onno_times_curated.csv")
print("Final ONOMAP rows:", len(onomap_df))
surname_only = onomap_df[onomap_df["Forename"].isna() | (onomap_df["Forename"].astype(str).str.strip() == "")]
print("Surname only retained:", len(surname_only))
print(surname_only.head(30))
print("Curated mass-casualty refs:", sorted(MASS_CASUALTY_REFS))

Mass-casualty victims loaded:
ref
2017-0206    30
2018-0262    71
2019-0332     8
Name: name_of_deceased, dtype: int64
Wrote: pfd_onno_times_curated.csv
Final ONOMAP rows: 5693
Surname only retained: 4
     Unique Identifier Forename    Surname
708         5474860571      NaN     Pether
2246        5711601195      NaN       Care
3647        9241999471      NaN    Withers
4116        1743138159      NaN  Van Tuyen
Curated mass-casualty refs: ['2017-0206', '2018-0262', '2019-0332']
